In [1]:
# Imports
# ---------

#import modin.pandas as pd
import sys
import pandas as pd
import numpy as np
import feather
import os
import gc
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pandas.plotting import register_matplotlib_converters
import matplotlib.ticker as ticker
from matplotlib.dates import DateFormatter
import matplotlib as mpl
from datetime import timezone

register_matplotlib_converters()

# File locations
# ----------------
the_gases = './oxaria/data/raw/1oxaria/json/gap_filling/'
pngs = './oxaria/data/pngs/gap_filling/'

In [2]:
# Load the df
# -------------
oxaria1_gases = pd.read_feather(
    the_gases+'oxaria1_gases_gf.ftr', use_threads=True).set_index(['tag', 'rec'])
oxaria1_gases_lt = oxaria1_gases.iloc[:, [0, 1, 2, 3, 16, 17, 19]]
del oxaria1_gases

KeyboardInterrupt: 

In [3]:
# Checking out the data types
# -----------------------------
print('\n'+'Loading Oxaria1 gases (no2 only)...\n')
oxaria1_gases_lt.info()


Loading Oxaria1 gases (no2 only)...

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 24173620 entries, ('scs-bgx-536', Timestamp('2020-01-04 09:45:52+0000', tz='UTC')) to ('scs-bgx-543', Timestamp('2021-02-28 23:59:54+0000', tz='UTC'))
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   val.no2.wev  float32
 1   val.no2.cnc  float32
 2   val.no2.aev  float32
 3   val.no2.wec  float32
 4   val.sht.hmd  float32
 5   val.sht.tmp  float32
 6   name         object 
dtypes: float32(6), object(1)
memory usage: 1.6+ GB


In [4]:
# Some cleaning incase have missed anything
# -------------------------------------------
oxaria1_gases_lt = oxaria1_gases_lt[~oxaria1_gases_lt.index.duplicated(
    keep='last')]
oxaria1_gases_lt = oxaria1_gases_lt.reset_index()
oxaria1_gases_lt = oxaria1_gases_lt.dropna(
    axis=0, subset=['tag']).set_index(['tag', 'rec'])
oxaria1_gases_lt.sort_index()
print('\n Cleaning duplicates & NAs in index (if any)...\n')


 Cleaning duplicates & NAs in index (if any)...



In [5]:
# Get a list of unique tags
# ---------------------------
tags = oxaria1_gases_lt.index.get_level_values(0).unique()
print('\n Unique tags loaded...\n'+tags)

Index(['\n Unique tags loaded...\nscs-bgx-536',
       '\n Unique tags loaded...\nscs-bgx-537',
       '\n Unique tags loaded...\nscs-bgx-538',
       '\n Unique tags loaded...\nscs-bgx-539',
       '\n Unique tags loaded...\nscs-bgx-540',
       '\n Unique tags loaded...\nscs-bgx-541',
       '\n Unique tags loaded...\nscs-bgx-542',
       '\n Unique tags loaded...\nscs-bgx-543'],
      dtype='object', name='tag')


In [6]:
# Define the start dates for stable operation
# ---------------------------------------------
start_dates = ['2020-09-25T00:00:00', '2020-08-01T00:00:00', '2020-06-05T00:00:00', '2020-01-25T00:00:00',
               '2020-05-01T00:00:00', '2020-03-05T00:00:00', '2020-02-06T00:00:00', '2020-12-07T00:00:00']
dates_list = [dt.datetime.fromisoformat(date).replace(
    tzinfo=timezone.utc) for date in start_dates]
dates_dict = dict(zip(tags, dates_list))
print('\n Applying variable start date filters of each sensor...\n'+str(dates_dict))


 Applying variable start date filters of each sensor...
{'scs-bgx-536': datetime.datetime(2020, 9, 25, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-537': datetime.datetime(2020, 8, 1, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-538': datetime.datetime(2020, 6, 5, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-539': datetime.datetime(2020, 1, 25, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-540': datetime.datetime(2020, 5, 1, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-541': datetime.datetime(2020, 3, 5, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-542': datetime.datetime(2020, 2, 6, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-543': datetime.datetime(2020, 12, 7, 0, 0, tzinfo=datetime.timezone.utc)}


In [7]:
# Select periods of stable operation from the df of all gases sesnor data
# -------------------------------------------------------------------------
print('\n Applying start date filters...\n')

tmp = []

for k, v in dates_dict.items():
    df = oxaria1_gases_lt.query('tag == @k & rec >= @v')
    tmp.append(df)
tmpdf = pd.concat(tmp)
tmpdf.info(null_counts=True)

# Apply empirical filters
# -------------------------

print("\n Applying sensible empirical filters...\n condition =  ((tmpdf['val.sht.tmp'] < -10.0) | \
             (tmpdf['val.sht.tmp'] > 35.0) | \
             (tmpdf['val.sht.hmd'] < 35.0))\n")

cols = ['val.no2.cnc']

condition = ((tmpdf['val.sht.tmp'] < -10.0) |
             (tmpdf['val.sht.tmp'] > 35.0) |
             (tmpdf['val.sht.hmd'] < 35.0))

for col in cols:
    tmpdf[col+'_1'] = np.where(condition, np.nan, tmpdf[col])

# Save to feather
# -----------------
print('\n Writing to  '+the_gases+'oxaria1_gases_stable_536_2feb21/.ftr\n')
tmpdf.reset_index().to_feather(the_gases+'oxaria1_gases_stable_536_2feb21/.ftr')
print('All done! \U0001F600')


 Applying start date filters...

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 18680104 entries, ('scs-bgx-536', Timestamp('2020-09-25 00:00:04+0000', tz='UTC')) to ('scs-bgx-543', Timestamp('2021-02-28 23:59:54+0000', tz='UTC'))
Data columns (total 7 columns):
 #   Column       Non-Null Count     Dtype  
---  ------       --------------     -----  
 0   val.no2.wev  18680104 non-null  float32
 1   val.no2.cnc  18680104 non-null  float32
 2   val.no2.aev  18680104 non-null  float32
 3   val.no2.wec  18680104 non-null  float32
 4   val.sht.hmd  18680104 non-null  float32
 5   val.sht.tmp  18680104 non-null  float32
 6   name         18680104 non-null  object 
dtypes: float32(6), object(1)
memory usage: 1.4+ GB

 Applying sensible empirical filters...
 condition =  ((tmpdf['val.sht.tmp'] < -10.0) |              (tmpdf['val.sht.tmp'] > 35.0) |              (tmpdf['val.sht.hmd'] < 35.0))


 Writing to  ./oxaria/data/raw/1oxaria/json/gap_filling/oxaria1_gases_stable_536_2feb21/.ftr

Al